
Using an external optimiser
---------------------------

`.optim_solve()` is flexible and the underlying `SciPy` algorithms can be configured by passing in keyword arguments, but in some cases it may be preferable to set up an external optimisation. This can be done by passing in the 

*'Ipopt (Interior Point OPTimizer, pronounced eye-pea-Opt) is a software package for large-scale nonlinear optimization. cyipopt is a python wrapper around Ipopt. It enables using Ipopt from the comfort of the great Python scripting language. Ipopt is available from the COIN-OR initiative, under the Eclipse Public License (EPL).'*

In [ ]:
from pysilsub.problems import SilentSubstitutionProblem as SSP

from cyipopt import minimize_ipopt


# Instantiate the problem class
ssp = SSP.from_package_data('STLAB_1_York')

# Define problem
ssp.ignore = ['rh']
ssp.modulate = ['mel']
ssp.minimize = ['sc', 'mc', 'lc']
ssp.target_contrast = 1.0
ssp.print_problem()

constraints = [
    {"type": "eq", "fun": ssp.silencing_constraint, "tol": 1e-05, "bounds": ssp.bounds * 2}
]
    
# Set up optimisation
solution = minimize_ipopt(
    fun=ssp.objective_function,
    x0=ssp.initial_guess_x0(),
    args=(),
    bounds=ssp.bounds * 2,
    constraints=constraints,
    tol=1e-08,
    options={b"print_level": 3, b"constr_viol_tol": 1e-02, b"max_iter": 100},
)

# Plot the solution
_ = ssp.plot_solution(solution.x)

************************************************************
*************** Silent Substitution Problem ****************
************************************************************
Device: STLAB_1 (binocular, left eye)
Observer: StandardColorimetricObserver(age=32, field_size=10)
Ignoring: ['rh']
Minimising: ['sc', 'mc', 'lc']
Modulating: ['mel']
Target contrast: [ 1.]
Background: None


Total number of variables............................:       20
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       20
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0



In [19]:
ssp.print_photoreceptor_contrasts(solution.x)

sc     0.062264
mc     0.080541
lc    -0.112253
rh     0.599552
mel    0.790417
dtype: float64
